<a href="https://colab.research.google.com/github/zahraa57/Machine-Learning-1-Course-Project/blob/main/feature_extractions_and_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AFLW2000 imge feature extraction**  

*  using mediapipe libraries to generate face mesh points.  
*  Extract poses from images mat files.
*  save a data frame contains Image name, generated points and correspoding pose parameters 






In [ ]:
%%capture
!pip install mediapipe

In [ ]:
#import libraries
import numpy as np
import os,cv2
import scipy.io as sio
import pandas as pd
import mediapipe as mp
import warnings
warnings.filterwarnings('ignore')

**scraping the data from the disk** 
* create a list of all images names and .mat files names
* create seprate data frame for the poses
* creat array of images to be ready for preprocessing



In [ ]:
images_list=[]
mat_list=[]
for filename in os.listdir("/content/drive/MyDrive/data/AFLW2000"):
    if   ".jpg" in filename :
        images_list.append(filename)
    elif ".mat" in filename:
        mat_list.append(filename)

In [ ]:
col_mat=["pitch", "yaw", "roll", "h", "v","t", "scale"]
mat_array=np.zeros((2000,7))
i=0
for name in mat_list:
    mat = sio.loadmat("/content/drive/MyDrive/data/AFLW2000/"+name)
    mat_array[i,:] = mat['Pose_Para'][0]
    i=i+1
df_mat=pd.DataFrame(mat_array, columns=col_mat)

In [ ]:
img_array=[]
for name in images_list:
    img = cv2.imread("/content/drive/MyDrive/data/AFLW2000/"+name)
    img_array.append(img)


create data frame for the feature points of images usig mediapipe </br> 
mediapipe generates 468 3d points , we will extract just x and y  </br>




In [ ]:
#two dfs each df for x axis points, the other for y axis points
cols_names_x=[]
for i in range(0,468):
    cols_names_x.append('x'+(str(i+1)))
cols_names_y=[]
for i in range(0,468):
    cols_names_y.append('y'+(str(i+1)))
    
df_x=pd.DataFrame(columns=cols_names_x)
df_y=pd.DataFrame(columns=cols_names_y)

data_x=np.zeros((2000,468))
data_y=np.zeros((2000,468))
ex=[]   # for image indicies that the code will not generate points for through processing

In [ ]:
for j in range(2000):
    image=img_array[j]
    #Face Mesh
    #rgb_image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    #Facial landmarks
    mp_face_mesh=mp.solutions.face_mesh.FaceMesh()
    result=mp_face_mesh.process(image)
    #height,width,_=image.shape 
    if result.multi_face_landmarks != None: 
        for facial_landmarks in result.multi_face_landmarks:
            #print(j)

            for i in range(0,468):
              pt1=facial_landmarks.landmark[i]
              data_x[j,i]=pt1.x
              data_y[j,i]=pt1.y
        # temp_x=pd.DataFrame(data_x,columns=cols_names_x)
        # temp_y=pd.DataFrame(data_y,columns=cols_names_y)
        # df_x=df_x.append(temp_x)
        # df_y=df_y.append(temp_y)
    else:
      ex.append(j)
df_x=pd.DataFrame(data_x,columns=cols_names_x)
df_y=pd.DataFrame(data_y,columns=cols_names_y)

In [ ]:
#concat x df with y df for all data
df=pd.concat([df_x,df_y],axis=1)

In [ ]:
#note zero rows --> coresponds to excluded images and needs to be filtered
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y459,y460,y461,y462,y463,y464,y465,y466,y467,y468
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000
3,0.539054,0.566098,0.538103,0.534302,0.567888,0.559214,0.529194,0.380115,0.520729,0.522354,...,0.584808,0.578603,0.588315,0.586334,0.590609,0.49318,0.497405,0.50034,0.485196,0.477088
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000


In [ ]:
#join features with corresponding poses
tabel=pd.concat([df,df_mat],axis=1)
tabel.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y466,y467,y468,pitch,yaw,roll,h,v,t,scale
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,-0.183631,-0.694458,0.613310,163.218369,210.338745,-95.068230,0.001065
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,-1.108159,-1.443685,1.129171,110.491158,203.644730,-11.035101,0.001160
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.003072,-1.330352,0.362599,111.676537,192.036057,-24.442087,0.001280
3,0.539054,0.566098,0.538103,0.534302,0.567888,0.559214,0.529194,0.380115,0.520729,0.522354,...,0.50034,0.485196,0.477088,-0.474087,-0.040907,0.019801,218.157730,280.057434,-113.012268,0.001805
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,-0.851643,1.205376,-0.556451,323.262878,216.088608,-27.155293,0.001088


In [ ]:
#add image_id for each row
pdname=pd.DataFrame(images_list,columns=["image_name"])
pdname.head()

,image_name
0,image03178.jpg
1,image03258.jpg
2,image03252.jpg
3,image03233.jpg
4,image03216.jpg


In [ ]:
data=pd.concat([pdname,tabel],axis=1)
data.head()

,image_name,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,y466,y467,y468,pitch,yaw,roll,h,v,t,scale
0,image03178.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,-0.183631,-0.694458,0.613310,163.218369,210.338745,-95.068230,0.001065
1,image03258.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,-1.108159,-1.443685,1.129171,110.491158,203.644730,-11.035101,0.001160
2,image03252.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.003072,-1.330352,0.362599,111.676537,192.036057,-24.442087,0.001280
3,image03233.jpg,0.539054,0.566098,0.538103,0.534302,0.567888,0.559214,0.529194,0.380115,0.520729,...,0.50034,0.485196,0.477088,-0.474087,-0.040907,0.019801,218.157730,280.057434,-113.012268,0.001805
4,image03216.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,-0.851643,1.205376,-0.556451,323.262878,216.088608,-27.155293,0.001088


In [ ]:
#filter zero featurs rows
data=data[np.count_nonzero(data.values, axis=1) > len(df.columns)-2]

In [ ]:
len(ex)

425

In [ ]:
#save data to csv
data.to_csv("/content/drive/MyDrive/data/data.csv",index=False)

In [ ]:
#ensure data retrival
d=pd.read_csv("/content/drive/MyDrive/data/data.csv")
d.head()

,image_name,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,y466,y467,y468,pitch,yaw,roll,h,v,t,scale
0,image03233.jpg,0.539054,0.566098,0.538103,0.534302,0.567888,0.559214,0.529194,0.380115,0.520729,...,0.500340,0.485196,0.477088,-0.474087,-0.040907,0.019801,218.157730,280.057434,-113.012268,0.001805
1,image03255.jpg,0.572712,0.561157,0.543249,0.500813,0.553606,0.532301,0.469587,0.337372,0.435622,...,0.460982,0.428940,0.423490,-0.331991,-0.280685,-0.057327,183.341934,244.958817,-92.797005,0.001587
2,image03236.jpg,0.503842,0.488899,0.498043,0.468165,0.485042,0.482925,0.480913,0.358855,0.474047,...,0.491338,0.447866,0.437037,0.036318,-0.447323,-0.115917,178.802704,194.897919,-97.699074,0.001242
3,image03222.jpg,0.536512,0.554113,0.538082,0.528365,0.555925,0.551769,0.535769,0.370772,0.532781,...,0.500151,0.475350,0.468136,-0.116700,-0.233745,0.095809,197.981903,211.594971,-102.926636,0.001377
4,image03266.jpg,0.531321,0.521944,0.523811,0.491416,0.518149,0.512927,0.499565,0.343950,0.489164,...,0.495773,0.460554,0.450567,0.093373,0.062462,-0.125822,235.452881,180.535965,-118.137459,0.001704


some shape check of data filtered and saved and excluded

In [ ]:
d.shape

(1575, 944)

In [ ]:
data.shape

(1575, 944)

In [ ]:
len(ex)

425